In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [45]:
# Visualization
import matplotlib.font_manager as fm

# 나눔 폰트 경로 설정
font_path = '/Library/Fonts/NanumGothic.ttf'

# 나눔 폰트 설정
fe = fm.FontEntry(
    fname=font_path,  # ttf 파일이 저장되어 있는 경로
    name='NanumGothic'  # 이 폰트의 원하는 이름 설정
)
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'})  # 폰트 설정
plt.rc('font', family='NanumGothic')

# data load
- slack 에 올라온 기준으로 처리합니다.
1. 초기 원본 데이터 : train_origin/test_origin (origin)
2. 두번째 나리님 데이터 : train_mid/test_mid (middle)
3. 세번째 윤설님 데이터 : train_final/test_final (final)

이 데이터에서 원본에서 거래유형 살리고, 제 데이터에서 급지를 10개로 추가해서 넣고, 최종 3번 데이터 에서 적절하게 취합하겠습니다. 

In [46]:
train_origin = pd.read_csv('./data/train.csv')
test_origin = pd.read_csv('./data/test.csv')

# train_mid = pd.read_csv('./data/train_final.csv')
# test_mid = pd.read_csv('./data/test_final.csv') 

train_final = pd.read_csv('./data/train_final_3.csv')
test_final = pd.read_csv('./data/test_final_3.csv')

In [47]:
display(train_origin.shape, train_origin.columns)

(1118822, 52)

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target'],
      dtype='object')

In [48]:
display(train_final.shape, train_final.columns)

(1118822, 39)

Index(['아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명', 'k-단지분류(아파트,주상복합등등)',
       'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
       'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', '건축면적', '주차대수',
       '기타/의무/임대/임의=1/2/3/4', '좌표X', '좌표Y', 'target', 'distance_score', '구',
       '동', '계약연도', '계약월', '급지', 'address', '신축여부', '이자율', '시공능력평가_상위여부',
       '대장아파트_거리'],
      dtype='object')

In [49]:
display(test_origin.shape, test_final.shape)

(9272, 51)

(9272, 39)

In [50]:
train_final['거래유형'] = train_origin['거래유형']
test_final['거래유형'] = test_origin['거래유형']

In [51]:
train_final['거래유형'] = train_final['거래유형'].replace('-', '중개거래')

In [52]:
display(train_final['거래유형'].value_counts(), test_final['거래유형'].value_counts()) 

거래유형
중개거래    1115722
직거래        3100
Name: count, dtype: int64

거래유형
중개거래    8852
직거래      420
Name: count, dtype: int64

In [53]:
#train_final.describe()
test_final.describe()

,전용면적,계약년월,계약일,층,건축년도,k-전체동수,k-전체세대수,k-연면적,k-주거전용면적,k-전용면적별세대현황(60㎡이하),...,좌표X,좌표Y,target,distance_score,계약연도,계약월,신축여부,이자율,시공능력평가_상위여부,대장아파트_거리
count,9272.000000,9272.000000,9272.000000,9272.000000,9272.000000,9272.000000,9272.000000,9.272000e+03,9272.000000,9272.000000,...,9272.000000,9272.000000,9272.0,9272.000000,9272.0,9272.000000,9272.000000,9272.000000,9272.000000,9272.000000
mean,75.414506,202307.786238,14.967429,10.021031,2003.034944,14.271948,1281.202060,2.263085e+05,101130.263967,476.493313,...,126.998508,37.552218,0.0,5.026639,2023.0,7.786238,0.318809,4.305435,0.244392,3.700806
std,29.343517,0.733229,8.802402,6.653698,11.179837,15.213184,1334.255853,2.685729e+06,111263.444755,587.592723,...,0.089031,0.053994,0.0,2.710025,0.0,0.733229,0.466040,0.025169,0.429749,2.091729
min,12.000000,202307.000000,1.000000,-3.000000,1961.000000,1.000000,74.000000,0.000000e+00,2338.000000,0.000000,...,126.798318,37.439151,0.0,0.000000,2023.0,7.000000,0.000000,4.280000,0.000000,0.081774
25%,59.700000,202307.000000,8.000000,5.000000,1996.000000,5.000000,455.875000,5.811963e+04,35286.000000,79.500000,...,126.920871,37.506397,0.0,3.000000,2023.0,7.000000,0.000000,4.280000,0.000000,2.049563
50%,76.570000,202308.000000,15.000000,9.000000,2003.000000,9.000000,836.000000,1.084600e+05,65367.000000,268.000000,...,127.017996,37.547659,0.0,5.000000,2023.0,8.000000,0.000000,4.310000,0.000000,3.546578
75%,84.960000,202308.000000,22.000000,14.000000,2012.000000,17.000000,1601.839286,1.990738e+05,119769.000000,660.000000,...,127.065604,37.590168,0.0,7.000000,2023.0,8.000000,1.000000,4.310000,0.000000,5.005584
max,301.470000,202309.000000,31.000000,65.000000,2023.000000,122.000000,9510.000000,1.477811e+08,734781.000000,3930.000000,...,127.179998,37.687725,0.0,10.000000,2023.0,9.000000,1.000000,4.350000,1.000000,10.237165


In [54]:
import plotly.express as px

house_data = train_final[['동', '전용면적', 'target', '계약연도']]
house_data['계약연도'] = house_data['계약연도'].astype(int)

# 'target/전용면적' 열 추가
house_data['평균가격'] = house_data['target'] / house_data['전용면적']

# '동'과 '계약연도'별 그룹화 후 평균 계산
grouped_data = house_data.groupby(['동', '계약연도'])['평균가격'].mean().reset_index()

# plotly로 그래프 그리기
fig = px.line(grouped_data, x='계약연도', y='평균가격', color='동', title='Average Price per Area by Year (2007-2023)')

# 그래프를 브라우저에서 표시
fig.show()

In [37]:
mean_prices = grouped_data.groupby('동')['평균가격'].mean().reset_index()

# 상위 20개 동 선택
top_20_dongs = mean_prices.nlargest(20, '평균가격')
top_20_data = grouped_data[grouped_data['동'].isin(top_20_dongs['동'])]

# plotly로 막대 그래프 그리기
fig = px.bar(top_20_data, x='동', y='평균가격', color='계약연도', barmode='group',
             title='Top 20 Dongs Average Price per Area',
             labels={'평균가격': 'Average Price per Area', '동': 'Dong', '계약연도': 'Year'})

# 그래프를 브라우저에서 표시
fig.show()

In [55]:
# 동 으로 급지를 나누지 말고 도로명으로 급지를 나눠본다.
display(train_final['도로명'].value_counts())

도로명
올림픽로 435       4322
솔샘로 174        3997
송파대로 345       3603
성북로4길 52       3487
양재대로 1340      3369
               ... 
오패산로 202          1
공항대로42길 13        1
양천로 77            1
구로동로8길 62-5       1
방울내로11길 38        1
Name: count, Length: 9165, dtype: int64

In [39]:
house_data = train_final[['도로명', '전용면적', 'target', '계약연도']]
house_data['계약연도'] = house_data['계약연도'].astype(int)

# 'target/전용면적' 열 추가
house_data['평균가격'] = house_data['target'] / house_data['전용면적']

# 2017년 이후 데이터만 필터링
house_data_filtered = house_data[house_data['계약연도'] >= 2017]

# '동'과 '계약연도'별 그룹화 후 평균 계산 (2017년 이후 데이터만 사용)
grouped_data = house_data_filtered.groupby(['도로명', '계약연도'])['평균가격'].mean().reset_index()

# '동'별 평균 계산 (2017년 이후 데이터 기반)
mean_prices = grouped_data.groupby('도로명')['평균가격'].mean().reset_index()

In [40]:
from scipy import stats

# '평균가격'을 기준으로 내림차순 정렬
mean_prices = mean_prices.sort_values(by='평균가격', ascending=False)

# 정규분포를 기반으로 10개의 범주로 나누기 위한 Z-점수 계산
mean_prices['Z_score'] = stats.zscore(mean_prices['평균가격'])

# Z-점수를 기반으로 5개의 범주로 나누기
mean_prices['급지'] = pd.cut(mean_prices['Z_score'], 
                           bins=[-np.inf, -0.84, -0.25, 0.25, 0.84, np.inf], 
                           labels=['G5', 'G4', 'G3', 'G2', 'G1'])

# 결과 출력
print(mean_prices)


               도로명         평균가격   Z_score  급지
6752  유엔빌리지길 80-36  5024.751554  8.971382  G1
4297       상도로 164  4926.781817  8.760969  G1
5541    압구정로79길 88  4519.558816  7.886364  G1
6751   유엔빌리지길 80-1  4166.861539  7.128865  G1
3031          만리재로  4029.623994  6.834116  G1
...            ...          ...       ...  ..
4028     불광로13가길 7   222.588623 -1.342370  G5
6061    연희로18안길 13   221.927162 -1.343790  G5
6531    우이천로30길 43   217.027768 -1.354313  G5
1080    구로동로22길 42   211.216969 -1.366793  G5
6817    은평터널로13길 4   143.143430 -1.512996  G5

[8739 rows x 4 columns]


In [41]:
# '급지' 열이 존재하는지 확인
if '급지' not in train_final.columns:
    train_final['급지'] = 'G5'
else:
    train_final = train_final.drop(columns=['급지']).merge(mean_prices[['도로명', '급지']], on='도로명', how='left')

In [42]:
display(train_final.shape, train_final.columns)

(1118822, 40)

Index(['아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명', 'k-단지분류(아파트,주상복합등등)',
       'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
       'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', '건축면적', '주차대수',
       '기타/의무/임대/임의=1/2/3/4', '좌표X', '좌표Y', 'target', 'distance_score', '구',
       '동', '계약연도', '계약월', 'address', '신축여부', '이자율', '시공능력평가_상위여부', '대장아파트_거리',
       '거래유형', '급지'],
      dtype='object')

In [19]:
display(train_final['급지'].value_counts())

급지
G1     211160
G5     176106
G6     168956
G7     118579
G4     116010
G3     115636
G2      94558
G8      81039
G9      22311
G10       129
Name: count, dtype: int64

In [20]:
# '급지' 열이 존재하는지 확인
if '급지' not in test_final.columns:
    train_final['급지'] = 'G5'
else:
    test_final = test_final.drop(columns=['급지']).merge(mean_prices[['도로명', '급지']], on='도로명', how='left')

In [22]:
display(test_final['급지'].value_counts())

급지
G1     2756
G5     1222
G2     1083
G6     1016
G3      995
G4      952
G7      658
G8      406
G9      101
G10       3
Name: count, dtype: int64

In [23]:
display(train_final.shape, train_final.columns)

(1118822, 40)

Index(['아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명', 'k-단지분류(아파트,주상복합등등)',
       'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
       'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', '건축면적', '주차대수',
       '기타/의무/임대/임의=1/2/3/4', '좌표X', '좌표Y', 'target', 'distance_score', '구',
       '동', '계약연도', '계약월', 'address', '신축여부', '이자율', '시공능력평가_상위여부', '대장아파트_거리',
       '거래유형', '급지'],
      dtype='object')

In [24]:
display(test_origin.shape, test_final.shape)

(9272, 51)

(9272, 40)

In [25]:
train_final.to_csv('./data/train_model_1.csv', index=False, encoding='utf-8-sig')
test_final.to_csv('./data/test_model_1.csv', index=False, encoding='utf-8-sig')